#### PHASE 3 OLAP queries

In [ ]:
#Run ETL process if needed
%run ../Phase2/ETLprocess.ipynb

In [ ]:
#pip install and import here
#%pip install psycopg2-binary
#%pip install SQLAlchemy

import pandas as pd
import configparser
import sqlalchemy 

In [ ]:
# Retrieve data from database
config = configparser.ConfigParser()
config.read('../settings.ini')
db_config = config['DB CONFIGURATION']
try:
    conn_string = "postgresql://"+db_config['USER']+":"+db_config['PASSWORD']+"@"+db_config['HOST']+"/"+db_config['DB_NAME']
    conn_string = conn_string.replace("'","") 
    db = sqlalchemy.create_engine(conn_string, pool_pre_ping=True)
    conn2 = db.connect()
    fact_table_df = pd.read_sql(sql='Fact_Table',con=conn2)
    cinfo_df = pd.read_sql(sql='ContextInfo_Dimension',con=conn2)
    pkd_entry_df = pd.read_sql(sql='PokedexEntry_Dimension',con=conn2)
    conn2.close()
except Exception as e:
    print("\nError:",e)
    

Rollup 

In [ ]:
# Roll up by ContextInfoKey
rollup_df = fact_table_df.groupby('ContextInfoKey').size().reset_index(name='count')
print(rollup_df)

Drill down

In [ ]:
df_tmp = pd.merge(fact_table_df, cinfo_df, on='ContextInfoKey', how='left')
df_fin = pd.merge(df_tmp, pkd_entry_df, on='DexEntryKey', how='left')
drill_down_df = pd.DataFrame()

# Drill down by generation
for generation, gen_group in df_fin.groupby('generation'):
    gen_count = len(gen_group)
    gen_group['generation_count'] = gen_count
    drill_down_df = pd.concat([drill_down_df, gen_group[['generation', 'generation_count']]])
drill_down_df.drop_duplicates(subset=['generation'], inplace= True)
print(drill_down_df)

Slice

In [ ]:
# Slicing the water type pokemon
slice_df = df_fin.loc[df_fin['type1'] == 'water'].reset_index()
print(slice_df)

Dice

In [ ]:
# Dice the pokemon have both water and poison type
dice_df = df_fin.loc[(df_fin['type1'] == 'water') & (df_fin['type2'] == 'poison')].reset_index()
print(dice_df)

Combining OLAP operations

In [ ]:
df_tmp = pd.merge(fact_table_df, cinfo_df, on='ContextInfoKey', how='left')
df_fin = pd.merge(df_tmp, pkd_entry_df, on='DexEntryKey', how='left')
print(df_fin)

In [ ]:
# Query 1:
# Query the fire pokemons in generation i Total attribute
generation1_df = df_fin.loc[df_fin['generation'] == 'generation-i']
cqueries_df_1 = generation1_df.loc[df_fin['type1'] == 'fire']
cqueries_df_1 = cqueries_df_1[['generation', 'name', 'type1', 'TOTAL']].reset_index()
print(cqueries_df_1)

# Query 2:
# Query the water pokemons that have variants
cqueries_df_2 = df_fin.loc[(df_fin['type1'] == 'water') & (df_fin['is_there_variant'] == True)]
cqueries_df_2 = cqueries_df_2[['generation', 'name', 'type1', 'is_there_variant']].reset_index()
print(cqueries_df_2)

# Query 3:
# Query the total number of grass pokemons in each generation
cqueries_df_3 = pd.DataFrame()
for generation, gen_group in df_fin.groupby('generation'):
    grass_df = gen_group.loc[df_fin['type1'] == 'grass'].copy()
    grass_count = len(grass_df)
    grass_df['count_grass'] = grass_count
    cqueries_df_3 = pd.concat([cqueries_df_3,grass_df[['generation','count_grass']]])
    cqueries_df_3.drop_duplicates(subset=['generation'], inplace= True)
print(cqueries_df_3) 


# Query 4:
# Query the slice of every legendary rank pokemon in generation in each generation
cqueries_df_4 = pd.DataFrame()
for generation, gen_group in df_fin.groupby('generation'):
    legendary = gen_group.loc[df_fin['rank'] == 'legendary'].copy()
    cqueries_df_4 = pd.concat([cqueries_df_4,legendary])
print(cqueries_df_4) 